# Building blocks for programming preconditioners (Unit 2.1.2)

In [14]:
import netgen.gui
from netgen.geom2d import unit_square
from ngsolve import *
%gui tk
from ngsolve.la import EigenValues_Preconditioner
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
Draw (mesh)

In [15]:
fes = H1(mesh, order=3, dirichlet="left|bottom")
u, v = fes.TnT()
a = BilinearForm(fes)
a += SymbolicBFI(grad(u)*grad(v) + u*v)
a.Assemble()
f = LinearForm(fes)
f += SymbolicLFI(1*v)
f.Assemble()
gfu = GridFunction(fes)

## Create a Jacobi-preconditioner

Let  $A=$ `a.mat` be the assembled matrix, which can be decomposed based on `FreeDofs` ($F$) the remainder ($D$), as in $\S$[1.3](unit-1.3-dirichlet/dirichlet.ipynb),

$$
A = \left( \begin{array}{cc} A_{FF} & A_{FD} \\ A_{DF} & A_{DD} \end{array} \right). 
$$

Then the matrix form of the **point Jacobi preconditioner** is

$$
J = \left( \begin{array}{cc} \text{diag}(A_{FF})^{-1} & 0  \\ 0 & I  \end{array} \right),
$$

which can be obtained in NGSolve using `CreateSmoother`:

In [16]:
preJpoint = a.mat.CreateSmoother(fes.FreeDofs())


NGSolve also gives us a facility to quickly check an estimate of the condition number of the preconditioned matrix by applying the Lanczos algorithm on the preconditioned system.


In [17]:
lams = EigenValues_Preconditioner(mat=a.mat, pre=preJpoint)
lams

 0.0146975
 0.0567754
 0.0951657
 0.112566
 0.14856
 0.198967
 0.255619
 0.364401
 0.446715
 0.533587
 0.638352
 0.752364
 0.836538
 0.933365
 1.04125
 1.16434
 1.24617
 1.36412
 1.48164
  1.5804
 1.69663
  1.8349
  1.9339
 2.04177
 2.13916
  2.2371
 2.29912
 2.38098
 2.42941
 2.50006
 2.51323
 2.54036
  2.8097

An estimate of the condition number 
$$
\kappa = \frac{\lambda_{\text{max}} }{ \lambda_{\text{min}}}
$$
is therefore given as follows:

In [18]:
max(lams)/min(lams)

191.16866539713283

One might wonder if we have gained anything by this point Jacobi preconditioning. What if we did not precondition at all?

Not preconditioning is the same as preconditioning by the identity operator on $F$-dofs. One way to realize this identity operator in NGSolve is through the projection into the space of free dofs (i.e., the space spanned by the shape functions corresponding to free dofs). NGSolve provides

```
Projector(mask, range)   # mask: bit array, range: bool
```

which projects into the space spanned by the shape functions of the degrees of freedom marked as range in mask.

In [19]:
preI = Projector(mask=fes.FreeDofs(), range=True)

*Note*  that another way to obtain the identity matrix in NGSolve is    
```
IdentityMatrix(fes.ndof, complex=False).
```

In [20]:
lams = EigenValues_Preconditioner(mat=a.mat, pre=preI)
max(lams)/min(lams)

1584.7214322156003

Clearly the point Jacobi preconditioner has reduced the condition number. 

We can use preconditioners within iterative solvers provided by NGSolve's `solvers` module (which has `MinRes`, `QMR` etc.) Here is an illustration of its use within CG, or the **conjugate gradient** solver: 

In [21]:
solvers.CG(mat=a.mat, pre=preJpoint, rhs=f.vec, sol=gfu.vec)
Draw(gfu)

it =  0  err =  0.0552223004818523
it =  1  err =  0.09382744174195903
it =  2  err =  0.10587866002038998
it =  3  err =  0.09033871153608698
it =  4  err =  0.10086392272140013
it =  5  err =  0.08517119355086314
it =  6  err =  0.08906575044224531
it =  7  err =  0.07684062913929476
it =  8  err =  0.05604717506189519
it =  9  err =  0.044720711033802436
it =  10  err =  0.03419741252004603
it =  11  err =  0.026824840496451088
it =  12  err =  0.02232298352240484
it =  13  err =  0.0200561993268448
it =  14  err =  0.014372559087884517
it =  15  err =  0.010772925966275763
it =  16  err =  0.009633149024762177
it =  17  err =  0.007301532183682153
it =  18  err =  0.004811737390492005
it =  19  err =  0.003253400552166538
it =  20  err =  0.0019025806942343491
it =  21  err =  0.0012857479602116594
it =  22  err =  0.0010097069801732912
it =  23  err =  0.0006971063184209092
it =  24  err =  0.0004415360534270253
it =  25  err =  0.0003131362679815841
it =  26  err =  0.00024593454

##  Gauss-Seidel smoothing

The *same* point Jacobi smoother object can also used to perform **point Gauss-Seidel** smoothing. One step of the classical Gauss-Seidel iteration is realized by the method `preJpoint.Smooth()`. Its well known that this iteration converges for matrices like $A$. Below we show how to use it as a linear iterative solver. 


In [22]:
help(preJpoint.Smooth)

Help on method Smooth in module ngsolve.la:

Smooth(...) method of ngsolve.la.Smoother instance
    Smooth(self: ngsolve.la.Smoother, x: ngsolve.la.BaseVector, b: ngsolve.la.BaseVector) -> None
    
    performs one step Gauss-Seidel iteration for the linear system A x = b



In [23]:
gfu.vec[:] = 0
res = f.vec.CreateVector()              # residual 
projres = f.vec.CreateVector()          # residual projected to freedofs
proj = Projector(fes.FreeDofs(), True)

for i in range(500):
    preJpoint.Smooth(gfu.vec, f.vec)    # one step of point Gauss-Seidel
    res.data = f.vec - a.mat*gfu.vec      
    projres.data = proj * res
    print ("it#", i, ", res =", Norm(projres))
Draw (gfu)

it# 0 , res = 0.08192679624402943
it# 1 , res = 0.07726762555864525
it# 2 , res = 0.07377040495808555
it# 3 , res = 0.0706222089560691
it# 4 , res = 0.06777582421551728
it# 5 , res = 0.06516770753241385
it# 6 , res = 0.06275317789329397
it# 7 , res = 0.06050089690216452
it# 8 , res = 0.058387526692409986
it# 9 , res = 0.056394947347622015
it# 10 , res = 0.054508699399513305
it# 11 , res = 0.05271702122673421
it# 12 , res = 0.05101020777328185
it# 13 , res = 0.04938015983450581
it# 14 , res = 0.04782005439140267
it# 15 , res = 0.046324095963871346
it# 16 , res = 0.044887324630715465
it# 17 , res = 0.04350546518198171
it# 18 , res = 0.04217480703984738
it# 19 , res = 0.04089210774233113
it# 20 , res = 0.03965451479597034
it# 21 , res = 0.03845950204060291
it# 22 , res = 0.03730481759400025
it# 23 , res = 0.03618844110638361
it# 24 , res = 0.035108548543330606
it# 25 , res = 0.03406348308437504
it# 26 , res = 0.03305173100814234
it# 27 , res = 0.03207190165593139
it# 28 , res = 0.03112271

it# 353 , res = 2.0287250505961437e-06
it# 354 , res = 1.9694640390639697e-06
it# 355 , res = 1.9119340988569535e-06
it# 356 , res = 1.856084663443388e-06
it# 357 , res = 1.8018666437994956e-06
it# 358 , res = 1.7492323847783378e-06
it# 359 , res = 1.6981356231389757e-06
it# 360 , res = 1.6485314469491583e-06
it# 361 , res = 1.6003762566392035e-06
it# 362 , res = 1.553627725729638e-06
it# 363 , res = 1.5082447644605394e-06
it# 364 , res = 1.464187483075384e-06
it# 365 , res = 1.4214171572982966e-06
it# 366 , res = 1.3798961940123723e-06
it# 367 , res = 1.3395880979087041e-06
it# 368 , res = 1.300457440165579e-06
it# 369 , res = 1.2624698265216736e-06
it# 370 , res = 1.22559186775363e-06
it# 371 , res = 1.1897911496269308e-06
it# 372 , res = 1.1550362048566067e-06
it# 373 , res = 1.1212964855603375e-06
it# 374 , res = 1.0885423358885244e-06
it# 375 , res = 1.0567449664483276e-06
it# 376 , res = 1.0258764287980762e-06
it# 377 , res = 9.95909590905561e-07
it# 378 , res = 9.668181132249977

## Implement a forward-backward GS preconditioner

The *same* point Jacobi smoother object is also able to perform a Gauss-Seidel iteration after reversing the ordering of the points, i.e., a **backward** Gauss-Seidel sweep. One can combine the forward and backward sweeps to construct a symmetric preconditioner, often called the **symmetrized Gauss-Seidel preconditioner**. This offers a good illustration of how to construct NGSolve preconditioners from within python. 

In [24]:
class SymmetricGS(BaseMatrix):
    def __init__ (self, smoother):
        super(SymmetricGS, self).__init__()
        self.smoother = smoother
    def Mult (self, x, y):
        y[:] = 0.0
        self.smoother.Smooth(y, x)
        self.smoother.SmoothBack(y,x)
    def Height (self):
        return self.smoother.height
    def Width (self):
        return self.smoother.width # this was height in original updated tutorial

In [28]:
preGS = SymmetricGS(preJpoint)
solvers.CG(mat=a.mat, pre=preGS, rhs=f.vec, sol=gfu.vec)
Draw (gfu)

it =  0  err =  0.0942142968361141
it =  1  err =  0.12242092674801885
it =  2  err =  0.08627911974342331
it =  3  err =  0.0542946368775426
it =  4  err =  0.026834953641369667
it =  5  err =  0.01444294653431131
it =  6  err =  0.00682816208895851
it =  7  err =  0.0029475389328079277
it =  8  err =  0.00098848748662747
it =  9  err =  0.00037122112200935724
it =  10  err =  0.00011770383684169352
it =  11  err =  4.729400969192553e-05
it =  12  err =  1.79595640434522e-05
it =  13  err =  7.73981616121218e-06
it =  14  err =  3.688537792536572e-06
it =  15  err =  1.7853513785616134e-06
it =  16  err =  6.793337817618374e-07
it =  17  err =  3.267271476258277e-07
it =  18  err =  1.0509766947753524e-07
it =  19  err =  5.263297661127212e-08
it =  20  err =  1.7518834053967592e-08
it =  21  err =  7.174942104447053e-09
it =  22  err =  2.5718606236788578e-09
it =  23  err =  7.674623733376325e-10
it =  24  err =  3.231702944081669e-10
it =  25  err =  7.901568725512919e-11
it =  26 

In [31]:
print(preGS.Width(), preGS.Height())

1096 1096


In [32]:
lams = EigenValues_Preconditioner(mat=a.mat, pre=preGS)
max(lams)/min(lams)

20.1666081867768

Note that the condition number now is better than that of the system preconditioned by point Jacobi.

## A Block Jacobi preconditioner

The point Jacobi preconditioner is based on inverses of 1 x 1 diagonal blocks.  Condition numbers can be improved by using larger blocks. It is possible to group dofs into blocks within python and construct an NGSolve preconditioner based on the blocks.

Here is an example that constructs vertex-based blocks.

In [41]:
blocks = []
freedofs = fes.FreeDofs()
for v in mesh.vertices:
    vdofs = set()
    for el in mesh[v].elements: # this only gives volume elements 
        vdofs |= set(d for d in fes.GetDofNrs(el) if freedofs[d]) # | is set union shorthand
    justone = False
    blocks.append (vdofs)
print (blocks)

[{866, 158, 159}, {13, 206, 207, 48, 145, 144, 882, 142, 143, 210, 211, 884}, {256, 257, 2, 901, 146, 147, 148, 21, 22, 149}, {65, 314, 919, 315, 917, 310, 150, 151, 154, 155, 311, 30}, {160, 161, 866, 867, 164, 165, 935, 40, 361, 360, 158, 159}, {869, 160, 161, 867, 164, 165, 868, 166, 40, 167, 41, 170, 171, 362, 363}, {868, 166, 167, 870, 41, 170, 171, 42, 172, 173, 871, 176, 177, 368, 369}, {375, 870, 872, 873, 42, 43, 172, 173, 176, 177, 178, 179, 182, 183, 374}, {380, 872, 874, 43, 44, 875, 178, 179, 381, 182, 183, 184, 185, 188, 189}, {194, 195, 386, 387, 874, 44, 876, 45, 877, 184, 185, 188, 189, 190, 191}, {194, 195, 196, 197, 200, 201, 392, 393, 876, 45, 46, 878, 879, 190, 191}, {196, 197, 200, 201, 202, 203, 204, 205, 398, 399, 878, 46, 47, 880, 881}, {202, 203, 204, 205, 206, 207, 144, 145, 404, 405, 47, 880, 48, 882, 883}, {13, 142, 143, 144, 145, 210, 211, 14, 208, 209, 212, 213, 217, 216, 410, 411, 48, 49, 884, 885, 886}, {13, 14, 15, 208, 209, 212, 213, 214, 215, 216, 21

`CreateBlockSmoother` can now take these blocks and construct a block Jacobi preconditioner.

In [42]:
blockjac = a.mat.CreateBlockSmoother(blocks)

lams = EigenValues_Preconditioner(mat=a.mat, pre=blockjac)
max(lams)/min(lams)

34.840427399307515

Multiplicative smoothers and its symmetrized version often yield better condition numbers in practice. We can apply the same code we wrote above for symmetrization (`SymmetricGS`) to the block smoother:

In [15]:
blockgs = SymmetricGS(blockjac)

lams = EigenValues_Preconditioner(mat=a.mat, pre=blockgs)
max(lams)/min(lams)

2.982606144898276

## Add a coarse grid correction

Dependence of the condition number on degrees of freedom can often be reduced by preconditioners that appropriately use a coarse grid correction.  It is also possible to experiment with coarse grid corrections using NGSolve's python interface. We now show how to precondition with a coarse grid correction made using the lowest order subspace of `fes`.

In the example below, note that we use `fes.GetDofNrs` again. Previously we used it with argument `el` of type `ElementId`, while now we use it with an argument `v` of type `MeshNode`.

In [16]:
vertexdofs = BitArray(fes.ndof)
vertexdofs[:] = False

for v in mesh.vertices:
    for d in fes.GetDofNrs(v):
        vertexdofs[d] = True
        
vertexdofs &= fes.FreeDofs()

print(vertexdofs)    # bit array, printed 50 chars/line

0: 00100000000001111111111111111110000000001111111111
50: 11111111111111111111111111111111111111111111111111
100: 11111111111111111111111111111111111100000000000000
150: 00000000000000000000000000000000000000000000000000
200: 00000000000000000000000000000000000000000000000000
250: 00000000000000000000000000000000000000000000000000
300: 00000000000000000000000000000000000000000000000000
350: 00000000000000000000000000000000000000000000000000
400: 00000000000000000000000000000000000000000000000000
450: 00000000000000000000000000000000000000000000000000
500: 00000000000000000000000000000000000000000000000000
550: 00000000000000000000000000000000000000000000000000
600: 00000000000000000000000000000000000000000000000000
650: 00000000000000000000000000000000000000000000000000
700: 00000000000000000000000000000000000000000000000000
750: 00000000000000000000000000000000000000000000000000
800: 00000000000000000000000000000000000000000000000000
850: 0000000000000000000000000000000000000000000000

Thus we have made a mask `vertexdofs` which reveals all free dofs associated to vertices. If these are labeled $c$ (and the remainder is labeled $f$), then the matrix $A$ can partitioned into 
$$
A = \left( \begin{array}{cc} A_{cc} & A_{cf} \\ A_{fc} & A_{ff} \end{array} \right). 
$$
The matrix `coarsepre` below represents
$$
\left( \begin{array}{cc} A_{cc}^{-1} & 0 \\ 0 & 0 \end{array} \right). 
$$

In [17]:
coarsepre = a.mat.Inverse(vertexdofs)

This matrix can be used for coarse grid correction. 

*Pitfall!*  Note that `coarsepre` is not appropriate as a preconditioner by itself as it has a large null space. You might get the wrong idea from the results of a Lanczos eigenvalue estimation:

In [18]:
EigenValues_Preconditioner(mat=a.mat, pre=coarsepre)

       1

But this result only gives the Laczos eigenvalue estimates on the *range* of the preconditioner. The preconditioned operator in this case is simply 
$$
\left( \begin{array}{cc} A_{cc}^{-1} & 0 \\ 0 & 0 \end{array} \right)
\left( \begin{array}{cc} A_{cc} & A_{cf} \\ A_{fc} & A_{ff} \end{array} \right)
 = 
 \left( \begin{array}{cc} I  & A_{cc}^{-1} A_{cf} \\ 0 & 0  \end{array} \right),
$$
which is a projection into the $c$-dofs. Hence its no surprise that Lanczos estimated the eigenvalues of this operator (on its range) to be just one. But this does not imply that the condition number of this preconditioned system is nice.

One well-known and correct way to combine the coarse grid correction with one of the previous smoothers is to combine them additively,  to get an **additive two-grid preconditioner** as follows.

In [19]:
twogrid = coarsepre + blockgs 

This addition of two operators (of type `BaseMatrix`) results in another operator, which is stored as an expression, to be evaluated only when needed.  The 2-grid preconditioner has a very good condition number.

In [20]:
EigenValues_Preconditioner(mat=a.mat, pre=twogrid)

 0.993081
 0.997768
 0.999961
 1.34206
 1.80314
 1.83857
 1.96023
  1.9828
 1.99987